<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [10]</a>'.</span>

# Demo 3 - Data Ingestion

This notebook reads the inference from ceph s3 storage for demo2 and will ingest these inference as a table to trino. These tables will be used for creating visualizations using Apache Superset.

In [1]:
import os
import pathlib
from dotenv import load_dotenv
import trino
import pandas as pd
import glob
import config
from src.data.s3_communication import S3Communication, S3FileType

### Injecting Credentials

In order to run this notebook, we need credentials to connect with S3 storage to retrieve data and the Trino server to create tables.

In an automated environment, the credentials can be specified in a pipeline's environment variables or through Openshift secrets.

For running the notebook in a local environment, we will define them as environment variables in a `credentials.env` file at the root of the project repository, and load them using dotenv. An example of what the contents of `credentials.env` could look like is shown below

```
# s3 credentials
S3_ENDPOINT=https://s3.us-east-1.amazonaws.com
S3_BUCKET=ocp-odh-os-demo-s3
AWS_ACCESS_KEY_ID=xxx
AWS_SECRET_ACCESS_KEY=xxx

# trino credentials
TRINO_USER=xxx
TRINO_PASSWD=xxx
TRINO_HOST=trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org
TRINO_PORT=443
```

In [2]:
# Load credentials
dotenv_dir = os.environ.get(
    "CREDENTIAL_DOTENV_DIR", os.environ.get("PWD", "/opt/app-root/src")
)
dotenv_path = pathlib.Path(dotenv_dir) / "credentials.env"
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path, override=True)

## Read Raw Data from S3

First, we will read some sample data from s3. We will format the column data types to ensure they can be understood by Trino, as well as rename the columns so that they are compatible with SQL naming conventions.

In [3]:
# init s3 connector
s3c = S3Communication(
    s3_endpoint_url=os.getenv("S3_ENDPOINT"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    s3_bucket=os.getenv("S3_BUCKET"),
)

In [4]:
if os.getenv("AUTOMATION"):
    if not os.path.exists(config.BASE_INFER_KPI_FOLDER):
        pathlib.Path(config.BASE_INFER_KPI_FOLDER).mkdir(parents=True, exist_ok=True)

    # Download a sample dataset file from s3
    s3c.download_files_in_prefix_to_dir(
        s3_prefix=config.BASE_INFER_KPI_S3_PREFIX,
        destination_dir=config.BASE_INFER_KPI_FOLDER
    )

In [5]:
all_files = glob.glob(str(config.BASE_INFER_KPI_FOLDER / "*.csv"))
list_of_files =  []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0).convert_dtypes().drop(columns=['Unnamed: 0'],axis=1)
    list_of_files.append(df)

preds_kpi = pd.concat(list_of_files, axis=0, ignore_index=True)

len_preds_kpi = len(preds_kpi)

# convert columns to specific data types
preds_kpi = preds_kpi.convert_dtypes().drop(['index'], axis=1, errors='ignore')
preds_kpi.head()

,pdf_name,kpi,kpi_id,answer,page,paragraph,source,score,no_ans_score,no_answer_score_plus_boost
0,sustainability-report-2019,In which year was the annual report or the sus...,<NA>,2019,3,This report focuses on the sustainability topi...,Text,12.819067,-11.38402,-26.38402
1,sustainability-report-2019,In which year was the annual report or the sus...,<NA>,2018,7,According to IPCC’s 1.5 C report from 2018 and...,Text,12.508747,-6.967487,-21.967487
2,sustainability-report-2019,In which year was the annual report or the sus...,<NA>,2019,26,Equinor Sustainability report 2019 High value ...,Text,12.427499,-9.680321,-24.680321
3,sustainability-report-2019,In which year was the annual report or the sus...,<NA>,2019,8,Equinor Sustainability report 2019Low carbon —...,Text,12.356201,-8.748014,-23.748014
4,sustainability-report-2019,What is the annual total production from coal?,<NA>,no_answer,<NA>,<NA>,Text,2.840456,<NA>,<NA>


In [6]:
# Author: Erik Erlandson <eje@redhat.com>

_p2smap = {"string": "varchar", "Float64": "double", "Int64": "bigint"}


def pandas_type_to_sql(pt):
    st = _p2smap.get(pt)
    if st is not None:
        return st
    raise ValueError("unexpected pandas column type '{pt}'".format(pt=pt))


# add ability to specify optional dict for specific fields?
# if column name is present, use specified value?
def generate_table_schema_pairs(df):
    ptypes = [str(e) for e in df.dtypes.to_list()]
    stypes = [pandas_type_to_sql(e) for e in ptypes]
    pz = list(zip(df.columns.to_list(), stypes))
    return ",\n".join(["    {n} {t}".format(n=e[0], t=e[1]) for e in pz])

In [7]:
# a way to examine the structure of a pandas data frame
preds_kpi.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pdf_name                    166 non-null    string 
 1   kpi                         166 non-null    string 
 2   kpi_id                      0 non-null      Int64  
 3   answer                      166 non-null    string 
 4   page                        139 non-null    Int64  
 5   paragraph                   139 non-null    string 
 6   source                      166 non-null    string 
 7   score                       166 non-null    Float64
 8   no_ans_score                139 non-null    Float64
 9   no_answer_score_plus_boost  139 non-null    Float64
dtypes: Float64(3), Int64(2), string(5)
memory usage: 13.9 KB


## Save Processed Data to S3

Now that our data is in a form ingestible by Trino, we will upload it back into our s3 bucket. This will be the data source for our Trino table.

In [8]:
# parquet has multiple options for appending or updating data
# including adding new files, or appending, sharding directory trees, etc
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0).convert_dtypes().drop(columns=['Unnamed: 0'],axis=1)
    ret = s3c.upload_df_to_s3(
        df,
        s3_prefix=config.BASE_INFER_KPI_TABLE_S3_PREFIX,
        s3_key=f"{os.path.basename(filename).split('.')[0]}.parquet",
        filetype=S3FileType.PARQUET,
        index=False,
    )
    print(ret['ResponseMetadata']['HTTPStatusCode'])

200
200


## Create a Table on Trino

Finally, we will create a table in our Trino database that uses the parquet files we uploaded in the previous section as the data source.

In [9]:
# use trino password env-var to hold token values
JWT_TOKEN = os.environ['TRINO_PASSWD']
conn = trino.dbapi.connect(
    host=os.environ['TRINO_HOST'],
    port=os.environ['TRINO_PORT'],
    user=os.environ['TRINO_USER'],
    http_scheme='https',
    auth=trino.auth.JWTAuthentication(JWT_TOKEN),
)
cur = conn.cursor()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [10]:
# generate a sql schema that will correspond to the data types
# of columns in the pandas DF
# to-do: add some mechanisms for overriding types, either here
# or on the pandas data-frame itself before we write it out
schema = generate_table_schema_pairs(preds_kpi)

tabledef = """create table if not exists osc_datacommons_dev.urgentem.infer_kpi(
{schema}
) with (
    format = 'parquet',
    external_location = 's3a://{s3_bucket}/{kpi_table_s3_prefix}/'
)""".format(
    schema=schema,
    s3_bucket=os.environ["S3_BUCKET"],
    kpi_table_s3_prefix=config.BASE_INFER_KPI_TABLE_S3_PREFIX,
)
# tables created externally may not show up immediately in cloud-beaver
cur.execute(tabledef)
cur.fetchall()

HttpError: error 401: b'Invalid credentials'

In [ ]:
## Check if infer_kpi table is there
cur.execute("select * from osc_datacommons_dev.urgentem.infer_kpi LIMIT 5")
cur.fetchall()[1]

# Conclusion

In this notebook, we read inference for KPI sustainability report, 2019 which follows the same format as the output of the KPI Inference model in Demo 2. After reading the report, we automatically infer the data schema from the report, preprocess it and create a table in trino that could be used for visualization in Apache Superset.